In [12]:
import pubchempy as pcp

# c_list = pcp.get_compounds("C6H10", "formula", record_type="3d")
search_name = "CH2O"
c_list = pcp.get_compounds(search_name, "name", record_type="3d")

# cid = 3034819
# c_list_clean = [pcp.Compound.from_cid(cid)]

for c in c_list:
    print(c.cid, c.iupac_name, c.synonyms)

c_list_clean = []
for c in c_list:
    c_list_clean.append(c)

for c in c_list_clean:
    print(c.cid, c.iupac_name, c.synonyms[0])

712 None ['formaldehyde', 'formalin', 'methanal', 'Paraformaldehyde', '50-00-0', 'formol', 'Oxomethane', 'Paraform', 'Methylene oxide', 'Formic aldehyde', 'Oxymethylene', 'Superlysoform', 'Lysoform', 'Methyl aldehyde', 'Fannoform', 'Formalith', 'Formalina', 'Methaldehyde', 'Oxomethylene', 'Morbicid', 'Karsan', '30525-89-4', 'Formaldehyd', 'Formaline', 'Aldehyde formique', 'FYDE', 'Formaldehyde, gas', 'Aldeide formica', 'Oplossingen', 'Formalin 40', 'Dormol', 'Rcra waste number U122', 'Aldehyd mravenci', 'Formalin-loesungen', 'Formalaz', 'Caswell No. 465', 'NCI-C02799', 'CH2O', 'UN 2209 (formalin)', 'Formaldehyde (gas)', 'UN 1198', 'Formaline [German]', 'Formalina [Italian]', 'Oplossingen [Dutch]', 'HCHO', 'CCRIS 315', 'FORMYL GROUP', 'HSDB 164', 'Fordor', 'NSC 298885', 'CHEBI:16842', 'Aldehyd mravenci [Czech]', 'Aldeide formica [Italian]', 'AI3-26806', 'Aldehyde formique [French]', 'BFV', 'Formalin-loesungen [German]', 'Formaldehyd [Czech, Polish]', 'EPA Pesticide Chemical Code 043001'

In [13]:
from pyscf.geomopt.geometric_solver import optimize
from pyscf import gto, scf, dft
import json
import copy
import json
from cadft.utils import rotate

with open("../cadft/utils/mol.json") as f:
    data = json.load(f)

dict_ = {}

for c in c_list_clean:
    # name = search_name
    # if name is None:
    name = c.synonyms[0]
    if name is None:
        print("No name for", c.cid)
        continue
    name = name.lower().replace(" ", "_")

    molecular = []
    for c_atom in c.atoms:
        atom = []
        atom.append(c_atom.element)
        atom.append(round(c_atom.x, 6))
        atom.append(round(c_atom.y, 6))
        if c_atom.z is None:
            atom.append(0.0)
        else:
            atom.append(round(c_atom.z, 6))
        molecular.append(atom)

    mol = gto.M(
        atom=molecular,
        basis="6-311g",
        # spin=1,
    )

    mdft = dft.RKS(mol)
    mdft.xc = "b3lyp"
    mdft.verbose = 4
    # mdft.max_cycle = 110
    # mdft.diis = None

    mol_eq = optimize(mdft, maxsteps=10, verbose=3)

    molecular_new = copy.deepcopy(molecular)
    for i, atom in enumerate(molecular_new):
        atom[1] = mol_eq.atom_coords(unit="ANG")[i][0]
        atom[2] = mol_eq.atom_coords(unit="ANG")[i][1]
        atom[3] = mol_eq.atom_coords(unit="ANG")[i][2]

    rotate(molecular_new)
    dict_[name] = molecular_new

data.update(dict_)

with open("../cadft/utils/mol.json", "w") as f:
    json.dump(data, f)

Create scanner for <class 'pyscf.grad.rks.Gradients'>
Create scanner for <class 'pyscf.dft.rks.RKS'>


geometric-optimize called with the following command line:
/home/dhem/anaconda3/envs/pyscf/lib/python3.12/site-packages/ipykernel_launcher.py --f=/home/dhem/.local/share/jupyter/runtime/kernel-v2-3566NI9giAqPoOSL.json

                                        ())))))))))))))))/                     
                                    ())))))))))))))))))))))))),                
                                *)))))))))))))))))))))))))))))))))             
                        #,    ()))))))))/                .)))))))))),          
                      #%%%%,  ())))))                        .))))))))*        
                      *%%%%%%,  ))              ..              ,))))))).      
                        *%%%%%%,         ***************/.        .)))))))     
                #%%/      (%%%%%%,    /*********************.       )))))))    
              .%%%%%%#      *%%%%%%,  *******/,     **********,      .))))))   
                .%%%%%%/      *%%%%%%,  **              *****


Geometry optimization cycle 1
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   O   0.612300   0.000000   0.000000    0.000000  0.000000  0.000000
   C  -0.612300   0.000000   0.000000    0.000000  0.000000  0.000000
   H  -1.200000   0.242600  -0.899800    0.000000  0.000000  0.000000
   H  -1.200000  -0.242400   0.899800    0.000000  0.000000  0.000000


******** <class 'pyscf.scf.hf.RKS_Scanner'> ********
method = RKS_Scanner
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
diis_damp = 0
SCF conv_tol = 1e-09
SCF conv_tol_grad = None
SCF max_cycles = 50
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /tmp/tmpu1xigqgn
max_memory 4000 MB (current use 1355 MB)
XC library pyscf.dft.libxc version 6.1.0
    S. Lehtola, C. Steigemann, M. J.T. Oliveira, and M. A.L. Marques.,  SoftwareX 7, 1–5 (2018)
XC functionals = b3lyp
    P. J. 

Step    0 : Gradient = 4.756e-03/7.711e-03 (rms/max) Energy = -114.4957996357
Rotator Norms:  0.0000
Hessian Eigenvalues: 4.50000e-02 5.00000e-02 5.00000e-02 ... 3.34877e-01 3.34925e-01 9.33739e-01
  Optimizer.step : Attempting full-size optimization step
sorted(eig) :  4.50000e-02  5.00000e-02  5.00000e-02 ...  3.34877e-01  3.34925e-01  9.33739e-01
      Trust radius Newton-Raphson step with v =    0.00000 : expect-deltaE = -0.00003, Internal-step = 0.01327
    InternalCoordinates.newCartesian converting internal to Cartesian step
      newCartesian Iter: 1 Err-dQ = 6.99062e-05 RMSD: 3.96444e-03 Damp: 1.00000e+00
      newCartesian Iter: 2 Err-dQ (Best) = 2.80149e-10 (6.99062e-05) RMSD: 2.01031e-05 Damp: 1.00000e+00 (Good)
      newCartesian Iter: 2 Cartesian coordinates obtained (rmsd = 2.010e-05 |dQ| = 2.801e-10)
sorted(eig) :  4.50000e-02  5.00000e-02  5.00000e-02 ...  3.34877e-01  3.34925e-01  9.33739e-01
      Trust radius Newton-Raphson step with v =    0.00000 : expect-deltaE =


Geometry optimization cycle 2
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   O   0.611631   0.000616  -0.002552   -0.000669  0.000616 -0.002552
   C  -0.615952  -0.000234   0.001694   -0.003652 -0.000234  0.001694
   H  -1.195575   0.242961  -0.901422    0.004425  0.000361 -0.001622
   H  -1.199608  -0.243142   0.902280    0.000392 -0.000742  0.002480


******** <class 'pyscf.scf.hf.RKS_Scanner'> ********
method = RKS_Scanner
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
diis_damp = 0
SCF conv_tol = 1e-09
SCF conv_tol_grad = None
SCF max_cycles = 50
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /tmp/tmpu1xigqgn
max_memory 4000 MB (current use 1322 MB)
XC library pyscf.dft.libxc version 6.1.0
    S. Lehtola, C. Steigemann, M. J.T. Oliveira, and M. A.L. Marques.,  SoftwareX 7, 1–5 (2018)
XC functionals = b3lyp
    P. J. 

...

Step    2 : Displace = 5.019e-03/6.147e-03 (rms/max) Trust = 1.414e-01 (+) Grad = 2.206e-03/3.244e-03 (rms/max) E (change) = -114.4958201117 (+6.053e-06) Quality = -0.305
Denoms: 9.128e-05 3.974e-05 Dots: 8.547e-01 8.630e-01 Eig-ratios: 1.00274e+00 ... 1.10660e+00
Rotator Norms:  0.0000
Hessian Eigenvalues: 4.51630e-02 4.99857e-02 5.00000e-02 ... 3.33617e-01 5.39775e-01 7.92407e-01
  Optimizer.step : Attempting full-size optimization step
sorted(eig) :  4.51630e-02  4.99857e-02  5.00000e-02 ...  3.33617e-01  5.39775e-01  7.92407e-01
      Trust radius Newton-Raphson step with v =    0.00000 : expect-deltaE = -0.00001, Internal-step = 0.00857
    InternalCoordinates.newCartesian converting internal to Cartesian step
      newCartesian Iter: 1 Err-dQ = 2.68733e-05 RMSD: 3.06045e-03 Damp: 1.00000e+00
      newCartesian Iter: 2 Err-dQ (Best) = 1.05336e-10 (2.68733e-05) RMSD: 7.52181e-06 Damp: 1.00000e+00 (Good)
      newCartesian Iter: 2 Cartesian coordinates obtained (rmsd = 7.522e-06 |dQ


Geometry optimization cycle 4
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   O   0.612412  -0.000083   0.000205   -0.001060  0.000661 -0.002977
   C  -0.616494   0.000122  -0.000148    0.000659 -0.000087  0.001959
   H  -1.197467   0.243012  -0.901450    0.002596 -0.000131 -0.000068
   H  -1.197087  -0.242844   0.901360   -0.002159 -0.000446  0.001106


******** <class 'pyscf.scf.hf.RKS_Scanner'> ********
method = RKS_Scanner
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
diis_damp = 0
SCF conv_tol = 1e-09
SCF conv_tol_grad = None
SCF max_cycles = 50
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /tmp/tmpu1xigqgn
max_memory 4000 MB (current use 1323 MB)
XC library pyscf.dft.libxc version 6.1.0
    S. Lehtola, C. Steigemann, M. J.T. Oliveira, and M. A.L. Marques.,  SoftwareX 7, 1–5 (2018)
XC functionals = b3lyp
    P. J. 

Step    3 : Displace = 2.624e-03/3.244e-03 (rms/max) Trust = 2.509e-03 (-) Grad = 1.618e-04/2.679e-04 (rms/max) E (change) = -114.4958346972 (-1.459e-05) Quality = 1.006
Denoms: 2.543e-05 2.551e-05 Dots: 8.708e-01 8.682e-01 Eig-ratios: 1.00250e+00 ... 9.93976e-01
Rotator Norms:  0.0000
Hessian Eigenvalues: 4.52759e-02 4.99863e-02 5.00000e-02 ... 3.33667e-01 5.38862e-01 7.87633e-01
  Optimizer.step : Attempting full-size optimization step
sorted(eig) :  4.52759e-02  4.99863e-02  5.00000e-02 ...  3.33667e-01  5.38862e-01  7.87633e-01
      Trust radius Newton-Raphson step with v =    0.00000 : expect-deltaE = -0.00000, Internal-step = 0.00117
    InternalCoordinates.newCartesian converting internal to Cartesian step
      newCartesian Iter: 1 Err-dQ = 4.25797e-07 RMSD: 3.33855e-04 Damp: 1.00000e+00
      newCartesian Iter: 1 Cartesian coordinates obtained (rmsd = 3.339e-04 |dQ| = 4.258e-07)
sorted(eig) :  4.52759e-02  4.99863e-02  5.00000e-02 ...  3.33667e-01  5.38862e-01  7.87633e-01
  


Geometry optimization cycle 5
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   O   0.612591   0.000053  -0.000022    0.000179  0.000136 -0.000227
   C  -0.616198  -0.000138  -0.000052    0.000296 -0.000260  0.000096
   H  -1.197259   0.243051  -0.901310    0.000208  0.000039  0.000140
   H  -1.197279  -0.242762   0.901354   -0.000192  0.000083 -0.000007


******** <class 'pyscf.scf.hf.RKS_Scanner'> ********
method = RKS_Scanner
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
diis_damp = 0
SCF conv_tol = 1e-09
SCF conv_tol_grad = None
SCF max_cycles = 50
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /tmp/tmpu1xigqgn
max_memory 4000 MB (current use 1323 MB)
XC library pyscf.dft.libxc version 6.1.0
    S. Lehtola, C. Steigemann, M. J.T. Oliveira, and M. A.L. Marques.,  SoftwareX 7, 1–5 (2018)
XC functionals = b3lyp
    P. J. 

Step    4 : Displace = 2.803e-04/3.261e-04 (rms/max) Trust = 3.549e-03 (+) Grad = 5.027e-05/8.628e-05 (rms/max) E (change) = -114.4958347713 (-7.413e-08) Quality = 0.714
Hessian Eigenvalues: 4.52759e-02 4.99863e-02 5.00000e-02 ... 3.33667e-01 5.38862e-01 7.87633e-01
Converged! =D

    #==========================================================================#
    #| If this code has benefited your research, please support us by citing: |#
    #|                                                                        |#
    #| Wang, L.-P.; Song, C.C. (2016) "Geometry optimization made simple with |#
    #| translation and rotation coordinates", J. Chem, Phys. 144, 214108.     |#
    #| http://dx.doi.org/10.1063/1.4952956                                    |#
    #==========================================================================#
    Time elapsed since start of run_optimizer: 6.753 seconds


In [14]:
from importlib.resources import files
from pathlib import Path
import os
import json

resource_path = files("cadft").joinpath("utils")
print(resource_path)
with open(
    Path(os.fspath(resource_path)) / "mol.json",
    "r",
    encoding="utf-8",
) as f:
    Mol = (json.load(f))
print(Mol.keys())


/home/dhem/workspace/2024.1/cadft/utils
dict_keys(['hexane', 'Cyclopentane', '2-methylpentane', '3-methylpentane', '2,2-dimethylbutane', '2,3-dimethylbutane', 'cyclohexane', '1-hexene', 'methylcyclopentane', '3,3-dimethyl-1-butene', '4-methyl-1-pentene', '2-methyl-1-pentene', '2-hexene', '2,3-dimethyl-2-butene', '3-hexene', '2-ethyl-1-butene', 'propylcyclopropane', '2,3-dimethyl-1-butene', '2-methyl-2-pentene', '4-methyl-2-pentene', '3-methyl-1-pentene', '2-pentene, 3-methyl-', '1-ethyl-1-methylcyclopropane', 'methane', 'ethane', 'ethylene', 'acetylene', 'cyclopentane', 'neopentane', 'isopentane', 'pentane', 'benzene', 'spiropentane', 'cyclobutane', 'cyclopropylmethyl', 'butyne', 'butadiene', 'butane', 'isobutane', 'propane', 'propyne', 'allene', 'cyclopropene', 'cyclopropane', 'propylene', 'bicyclobutane', 'cyclohexene', 'adamantane', 'bicyclo[2.2.2]octane', 'octane', 'ethyl-openshell', 'methyl-openshell', 'bicyclo-hexene', 'methanol', 'formaldehyde'])


In [15]:
print(list(Mol.keys())[-2:])

['methanol', 'formaldehyde']
